In [29]:
def swapTokensForExactTokens(toAmount, path):
    print('Swap from token {} to {} token {}'.format(path[0], toAmount, path[1]))

def swapExactTokensForTokens(fromAmount, path):
    print('Swap from {} token {} to token {}'.format(fromAmount, path[0], path[1]))

swapTokensForExactTokens(100, ['fromToken', 'toToken'])
swapExactTokensForTokens(1, ['fromToken', 'toToken'])

Swap from token fromToken to 100 token toToken
Swap from 1 token fromToken to token toToken


In [89]:
def getSellAmount(nIn, rIn, rOut):
    if (nIn == 0):
        return 0
    else:
        aInWithFee = nIn * 997
        numerator = aInWithFee * rOut
        denominator = rIn * 1000 + aInWithFee
        return numerator / denominator

getSellAmount(nIn=2.001066858179911, rIn=10000, rOut=200000)

39.893314181976834

In [99]:
# Swap to repay all debts
def swapToRepayAll(ra, rb, da, db, na, nb):
    leftA = na
    leftB = nb

    if (na > da and nb < db):
        # Swap A to (db - nb) B
        swapTokensForExactTokens(db - nb, ['A', 'B'])
        leftB += db - nb
        leftA -= getSellAmount(nIn=db-nb, rIn=rb, rOut=ra)  # Not exactly

    elif (nb > db and na < da):
        # Swap B to (da - na) A
        swapTokensForExactTokens(da - na, ['B', 'A'])
        leftA += da - na
        leftB -= getSellAmount(nIn=da-na, rIn=ra, rOut=rb)  # Not exactly

    elif (na < da and nb < db):
        print('Not enough token to repay debt, don\'t swap')
    else:
        print('Can repay without swap, don\'t swap')
    
    assert leftA >= da and leftB >= db, "require repay all debts"

    return leftA, leftB


print(swapToRepayAll(ra=10000, rb=200000, da=1, db=100, na=3, nb=80))          # A is much more
print(swapToRepayAll(ra=10000, rb=200000, da=1, db=100, na=0.5, nb=300))       # B is much more
# swapToRepayAll(ra=10000, rb=200000, da=1, db=100, na=0.5, nb=50)        # Both less than debts, should assert error
print(swapToRepayAll(ra=10000, rb=200000, da=1, db=100, na=2, nb=200))         # Both more than debts

Swap from token A to 20 token B
(2.003099390990718, 100)
Swap from token B to 0.5 token A
(1.0, 290.03049697972557)
Can repay without swap, don't swap
(2, 200)


In [77]:
import math

# Swap A to B with the debts ratio
def swapAToBWithDebtsRatio(ra, rb, da, db, na, nb):
    assert na * db > nb * da, "na / da should lager than nb / db"

    if (da == 0):
        return na

    part1 = nb * da / db - na
    part2 = 1000 * ra / 997
    part3 = da * rb / db

    b = part1 + part2 + part3
    c = part1 * part2


    return (-b + math.sqrt(b * b - 4 * c)) / 2

print(swapAToBWithDebtsRatio(ra=10000, rb=200000, da=1, db=100, na=3, nb=60))       # Swap 2 A to B
print(swapAToBWithDebtsRatio(ra=10000, rb=200000, da=0, db=100, na=3, nb=60))       # Swap 3 A to B
print(swapAToBWithDebtsRatio(ra=10000, rb=200000, da=0, db=100, na=0.01, nb=0))     # Swap 0.01 A to B
# print(swapAToBWithDebtsRatio(ra=10000, rb=200000, da=1, db=100, na=1, nb=200))      # A is less, should assert error
# print(swapAToBWithDebtsRatio(ra=10000, rb=200000, da=0, db=0, na=1, nb=200))        # da and db are both 0, should assert error

2.001066858179911
3
0.01


In [28]:
(60 + 39.893314181976834) / (3 - 2.001066858179911) # Should equal to 100 / 1 (db / da)

99.9999999999679

In [101]:
def repayDebts(ra=10000, rb=200000, da=1, db=100, na=3, nb=60):
    # If no debt, don't swap.
    if (da == 0 and db == 0):
        print("No debts")
        leftA = na
        leftB = nb
    
    # If current amount can repay all debts (Swap all tokens to one and compare):
    elif (getSellAmount(na, ra, rb) + nb > getSellAmount(da, ra, rb) + db and
          getSellAmount(nb, rb, ra) + na > getSellAmount(db, rb, ra) + da):

        print("Repay all debts: ", end='')
        leftA, leftB = swapToRepayAll(ra, rb, da, db, na, nb)

    # Otherwise, exchange tokens according to two debts amount ratio
    else:
        print("Repay with debts ratio: ", end='')
        if (na * db > nb * da):
            # na/da > nb/db, swap A to B

            amount = swapAToBWithDebtsRatio(ra, rb, da, db, na, nb)
            amount = na if amount > na else amount
            swapExactTokensForTokens(amount, ['A', 'B'])

            leftA = na - amount
            leftB = nb + getSellAmount(amount, ra, rb)

        elif (na * db < nb * da):
            # Otherwise, swap B to A

            amount = swapAToBWithDebtsRatio(rb, ra, db, da, nb, na)
            amount = nb if amount > nb else amount
            swapExactTokensForTokens(amount, ['B', 'A'])

            leftA = na + getSellAmount(amount, rb, ra)
            leftB = nb - amount

        else:
            print('na/da == nb/db, don\'t swap')

    return leftA, leftB

print(" Repay all")
print(" - Two debts")
print(repayDebts(ra=10000, rb=200000, da=1, db=100, na=4, nb=60))      # Swap A to 40 B
print(repayDebts(ra=10000, rb=200000, da=1, db=100, na=0, nb=120.4))   # Swap B to 1 A
print(repayDebts(ra=10000, rb=200000, da=1, db=100, na=2, nb=200))     # Enough, don't swap

print("\n - One debts")
print(repayDebts(ra=10000, rb=200000, da=0, db=100, na=4, nb=60))      # Swap A to 40 B
print(repayDebts(ra=10000, rb=200000, da=0, db=100, na=4, nb=100))     # Enough, don't swap
print(repayDebts(ra=10000, rb=200000, da=1, db=0, na=0, nb=21))        # Swap B to 1 A
print(repayDebts(ra=10000, rb=200000, da=1, db=0, na=1, nb=20))        # Enough, don't swap

print("\n Repay according to two debts amount ratio")
print(" - Two debts")
print(repayDebts(ra=10000, rb=200000, da=1, db=100, na=3, nb=60))      # Swap about 2 A to B
print(repayDebts(ra=10000, rb=200000, da=1, db=100, na=0.5, nb=110))   # Swap about 10 B to A

print("\n - One debts")
print(repayDebts(ra=10000, rb=200000, da=1, db=0, na=0.5, nb=10))       # Swap about 10 B to A
print(repayDebts(ra=10000, rb=200000, da=1, db=0, na=0, nb=20))         # Swap about 20 B to A
print(repayDebts(ra=10000, rb=200000, da=0, db=20, na=0.5, nb=10))      # Swap about 0.5 A to B
print(repayDebts(ra=10000, rb=200000, da=0, db=20, na=1, nb=0))         # Swap about 1 to B

# No debts
print("\n No debts")
print(repayDebts(ra=10000, rb=200000, da=0, db=0, na=1, nb=0))         # No debts, don't swap

 Repay all
 - Two debts
Repay all debts: Swap from token A to 40 token B
(2.006397524333648, 100)
Repay all debts: Swap from token B to 1 token A
(1, 100.46198781981437)
Repay all debts: Can repay without swap, don't swap
(2, 200)

 - One debts
Repay all debts: Swap from token A to 40 token B
(2.006397524333648, 100)
Repay all debts: Can repay without swap, don't swap
(4, 100)
Repay all debts: Swap from token B to 1 token A
(1, 1.061987819814366)
Repay all debts: Can repay without swap, don't swap
(1, 20)

 Repay according to two debts amount ratio
 - Two debts
Repay with debts ratio: Swap from 2.001066858179911 token A to token B
(0.9989331418200891, 99.89331418197683)
Repay with debts ratio: Swap from 10.025479962816462 token B to token A
(0.9997452003717189, 99.97452003718354)

 - One debts
Repay with debts ratio: Swap from 10 token B to token A
(0.998475151013722, 0)
Repay with debts ratio: Swap from 20 token B to token A
(0.9969006090092818, 0)
Repay with debts ratio: Swap from 0.

In [114]:
# Whithdraw money.
def execute(ra, rb, da, db, na, nb, rate, whichWantBack):
    if rate == 0:
        print("Rate is 0, do nothing", end='\n\n')
        return

    tempNa = na * rate / 10000
    tempNb = nb * rate / 10000

    # If want token back, repay debts with given rate.
    # Otherwise, use all withdrawn money to repay debts.
    if (whichWantBack != 3):
        tempDa = da * rate / 10000
        tempDb = db * rate / 10000
    else:
        tempDa = da
        tempDb = db

    leftA, leftB = repayDebts(ra, rb, tempDa, tempDb, tempNa, tempNb)

    # Start to repay
    if (leftA >= tempDa):
        print("Repay A with amount {}, back amount {}, left debts {}".format(tempDa, leftA-tempDa, da-tempDa))
    else:
        print("Repay A with amount {}, back amount {}, left debts {}".format(leftA, 0, da-leftA))

    if (leftB >= tempDb):
        print("Repay B with amount {}, back amount {}, left debts {}".format(tempDb, leftB-tempDb, db-tempDb))
    else:
        print("Repay B with amount {}, back amount {}, left debts {}".format(leftB, 0, db-leftB))
    
    print()

# Just repay
execute(ra=10000, rb=200000, da=1, db=100, na=6, nb=120, rate=0, whichWantBack=3)   # Rate is 0
execute(ra=10000, rb=200000, da=1, db=100, na=6, nb=120, rate=1000, whichWantBack=3)   #
execute(ra=10000, rb=200000, da=1, db=100, na=6, nb=120, rate=5000, whichWantBack=3)   # Swap about 2 A to B
execute(ra=10000, rb=200000, da=1, db=100, na=6, nb=120, rate=10000, whichWantBack=3)   #

Rate is 0, do nothing

Repay with debts ratio: Swap from 0.40020275464939914 token A to token B
Repay A with amount 0.19979724535060084, back amount 0, left debts 0.8002027546493992
Repay B with amount 19.979724534987316, back amount 0, left debts 80.02027546501269

Repay with debts ratio: Swap from 2.001066858179911 token A to token B
Repay A with amount 0.9989331418200891, back amount 0, left debts 0.0010668581799109234
Repay B with amount 99.89331418197683, back amount 0, left debts 0.10668581802316623

Repay all debts: Can repay without swap, don't swap
Repay A with amount 1, back amount 5.0, left debts 0
Repay B with amount 100, back amount 20.0, left debts 0



In [121]:
# Only repay debt with the input rate
execute(ra=10000, rb=200000, da=1, db=200, na=6, nb=120, rate=1000, whichWantBack=2)   #
execute(ra=10000, rb=200000, da=1, db=100, na=6, nb=120, rate=5000, whichWantBack=2)   #
execute(ra=10000, rb=200000, da=1, db=100, na=6, nb=120, rate=10000, whichWantBack=2)   #

Repay all debts: Swap from token A to 8.0 token B
Repay A with amount 0.1, back amount 0.10121590350976803, left debts 0.9
Repay B with amount 20.0, back amount 0.0, left debts 180.0

Repay all debts: Can repay without swap, don't swap
Repay A with amount 0.5, back amount 2.5, left debts 0.5
Repay B with amount 50.0, back amount 10.0, left debts 50.0

Repay all debts: Can repay without swap, don't swap
Repay A with amount 1.0, back amount 5.0, left debts 0.0
Repay B with amount 100.0, back amount 20.0, left debts 0.0

